In [1]:
import pandas as pd
from geopy import distance
import itertools
import numpy as np

In [2]:
def calculate_distance(p1,p2):
    return distance.distance(p1,p2).km

Compare GMaps and Web Scraping methods

In [18]:
banch_mark_df=pd.read_csv('correct_nl.csv',index_col=0)

In [30]:
benchmark_set=set([tuple(row[1].to_list()) for row in banch_mark_df.drop_duplicates().iterrows()])

In [31]:
benchmark_set=set([i for i in benchmark_set if i[1]>50 and i[2]>0])

In [32]:
for i in benchmark_set:
    if len(i)<4:
        print(i)

In [23]:
len(benchmark_set)

278

merge TRUE dataset with threshold  100m

In [36]:
previous_len=None
tmp_set=benchmark_set.copy()
while(previous_len!=len(tmp_set)):
    previous_len=len(tmp_set)
    flag=0
    print('---')
    for i in tmp_set:
        #print('--')
        
        for j in tmp_set:
            #print('--')
            #print(i,j)
            ##if they are close within 100 meters, then merge them
            if i!=j and calculate_distance((i[1],i[2]),(j[1],j[2]))<0.1:
                print('--')
                print(i,j)
                #print(calculate_distance(i,j))
                # if they have same address, keep it, or concat them
                if (i[3]==j[3]):
                    tmp_address=i[3]
                else:
                    tmp_address=i[3]+" and "+j[3]
                    
                # if they have same name, keep it, or concat them
                if (i[0]==j[0]):
                    tmp_name=i[0]
                else:
                    tmp_name=i[0]+" and "+j[0]
                
                # merging by mean of geo location
                k=(i[0]+" and "+j[0],(i[1]+j[1])/2,(i[2]+j[2])/2,tmp_address)
                tmp_set.remove(i)
                tmp_set.remove(j)
                tmp_set.add(k)
                flag=1
            if flag:
                break
        if flag:
            break

---
--
('Saffier 16  Julianadorp,  Noord-Holland 1761 VD', 52.86136679999999, 4.819212, 'Afdeling Hollands Kroon') ('Saffier 16  Julianadorp,  Noord-Holland 1761 VD', 52.8613106, 4.8191593, 'Afdeling Hollands Kroon')
---
--
('Lichtenbergerstraat 7 b  Landgraaf,  Limburg 6371 AS', 50.9037859, 6.0194126, 'Afdeling Landgraaf') ('Lichtenbergerstraat 7 b  Landgraaf,  Limburg 6371 AS', 50.9038888, 6.0196146, 'Afdeling Landgraaf')
---
--
('De Sterke Arm 10  Veenendaal,  Utrecht 3901 ER', 52.0221935, 5.556435, 'Afdeling Veenendaal') ('De Sterke Arm 10  Veenendaal,  Utrecht 3901 ER', 52.021980400000004, 5.55629, 'Afdeling Veenendaal')
---
--
('Maltezerlaan 3  Harderwijk,  Gelderland 3842 AZ', 52.343744799999996, 5.6239057, 'Afdeling Harderwijk') ('Maltezerlaan 3  Harderwijk,  Gelderland 3842 AZ', 52.343805100000004, 5.624671299999999, 'Afdeling Harderwijk')
---
--
('Oude Dieze 12  S-Hertogenbosch,  Noord-Brabant 5211 KV', 51.6856811, 5.3081307, 'Afdeling s-Hertogenbosch e.o.') ('Oude Dieze 12  

--
('Bergweg 4  Venray,  Limburg 5801 EG', 51.5263244, 5.9674363, 'Afdeling Venray') ('Bergweg 4  Venray,  Limburg 5801 EG Venray', 51.5263361, 5.9675671999999995, 'Afdeling Venray')
---
--
('Hofkampstraat 102  Almelo,  Overijssel 7607 NJ', 52.353057899999996, 6.673963400000001, 'Afdeling Almelo') ('Hofkampstraat 102  Almelo,  Overijssel 7607 NJ', 52.3532884, 6.6735415, 'Afdeling Almelo')
---
--
('Oranjewal 28   Dokkum,  Friesland 9101 JV ', 53.3280322, 5.9984980000000006, 'District Fryslân - Werkgroep Dokkum') ('Oranjewal 28  Dokkum,  Friesland 9101 JV', 53.327965, 5.998493700000001, 'District Fryslân - Werkgroep Dokkum')
---
--
('Buiten de Veste 10  Schoonhoven,  Zuid-Holland 2871 GE', 51.948413, 4.8452134000000004, 'Afdeling Krimpenerwaard') ('Buiten de Veste 10  Schoonhoven,  Zuid-Holland 2871 GE', 51.9484035, 4.8452331, 'Afdeling Krimpenerwaard')
---
--
('Boerhaavestraat 15 A  Weert,  Limburg 6001 AV', 51.2555536, 5.711226, 'Afdeling Weert e.o.') ('Boerhaavestraat 15 A  Weert,  Li

In [37]:
len(tmp_set)

210

In [39]:
merged_true=pd.DataFrame(list(tmp_set))
merged_true.columns=["name",'lat','lon','address']

In [51]:
Gmap_df=pd.read_csv('GMaps_Netherlands.csv',index_col=0)

In [53]:
Gmap_df_location=Gmap_df[['place_name','place_lat','place_long','place_address']]

In [54]:
#Gmap_df_location=Gmap_df[["place_lat","place_long"]]

In [55]:
Gmap_set=set([tuple(row[1].to_list()) for row in Gmap_df_location.drop_duplicates().iterrows()])

In [56]:
Gmap_set

{('Beverwijk, Rode Kruis Ziekenhuis',
  52.47916300000001,
  4.6500720000000015,
  '1942 LJ Beverwijk, Netherlands'),
 ('Helipad Red Cross Hospital',
  52.479110600000006,
  4.6515328,
  'Vondellaan 13, 1942 LE Beverwijk, Netherlands'),
 ('Het Haagse Rode Kruis',
  52.093712200000006,
  4.3084506,
  'Koninginnegracht 92, 2514 AK Den Haag, Netherlands'),
 ('Het NEderlandse Rode Kruis -Afdeling Niedorp',
  52.76852299999999,
  4.8840294,
  'Weereweg 31, 1732 LJ Lutjewinkel, Netherlands'),
 ('Het Nederlands Rode Kruis Gemeente Heumen',
  51.7798336,
  5.8532885000000014,
  'Rijksweg 144A, 6581 ES Malden, Netherlands'),
 ('Het Nederlandse Rode Kruis',
  51.242488200000004,
  5.568671399999999,
  'Stationsweg 55, 6024 BK Budel-Dorplein, Netherlands'),
 ('Het Nederlandse Rode Kruis',
  51.33904769999999,
  6.0588282,
  'Rinkesfort 15, 5993 PV Maasbree, Netherlands'),
 ('Het Nederlandse Rode Kruis',
  51.4020406,
  5.7059291,
  'Heiveldsestraat 21, 5711 PB Someren, Netherlands'),
 ('Het Neder

In [57]:
previous_len=None
tmp_set=Gmap_set.copy()
while(previous_len!=len(tmp_set)):
    previous_len=len(tmp_set)
    flag=0
    print('---')
    for i in tmp_set:
        #print('--')
        
        for j in tmp_set:
            #print('--')
            #print(i,j)
            ##if they are close within 100 meters, then merge them
            if i!=j and calculate_distance((i[1],i[2]),(j[1],j[2]))<0.1:
                print('--')
                print(i,j)
                #print(calculate_distance(i,j))
                # if they have same address, keep it, or concat them
                if (i[3]==j[3]):
                    tmp_address=i[3]
                else:
                    tmp_address=i[3]+" and "+j[3]
                    
                # if they have same name, keep it, or concat them
                if (i[0]==j[0]):
                    tmp_name=i[0]
                else:
                    tmp_name=i[0]+" and "+j[0]
                
                # merging by mean of geo location
                k=(i[0]+" and "+j[0],(i[1]+j[1])/2,(i[2]+j[2])/2,tmp_address)
                tmp_set.remove(i)
                tmp_set.remove(j)
                tmp_set.add(k)
                flag=1
            if flag:
                break
        if flag:
            break

---
--
('Rode Kruis Voorne Putten', 51.897403000000004, 4.166385, 'Plantageweg 4, 3232 LE Brielle, Netherlands') ('The Netherlands Red Cross Society', 51.89742099999999, 4.166371, '3230 AE Brielle, Netherlands')
---
--
('Verenigingskantoor Nederlandse Rode Kruis', 52.08186729999999, 4.34682, 'Anna van Saksenlaan 50, 2593 HT Den Haag, Netherlands') ('The Netherlands Red Cross', 52.0818604, 4.346826, 'Anna van Saksenlaan 50, 2593 HT Den Haag, Netherlands')
---
--
('Rode Kruis District Kennemerland', 52.41446989999999, 4.6522078, 'Vergierdeweg 52, 2025 TK Haarlem, Netherlands') ('Rode Kruis Haarlem e.o.', 52.41453370000001, 4.6521301, 'Vergierdeweg 52, 2025 TK Haarlem, Netherlands')
---
--
('Het Nederlandse Rode Kruis', 51.6078748, 5.5467325, 'Evertsenstraat 1, 5463 HM Veghel, Netherlands') ('Rode Kruis Afdeling Veghel eo', 51.6078748, 5.5467325, 'Evertsenstraat 9, 5463 HM Veghel, Netherlands')
---
--
('Rode Kruis Ziekenhuis', 52.47918539999999, 4.6509404000000005, 'Vondellaan 13, 1942 LE

In [58]:
merged_Gmap=pd.DataFrame(list(tmp_set))
merged_Gmap.columns=['place_name','place_lat','place_long','place_address']

In [59]:
web_scrapying_df=pd.read_csv("nl_result_full_name.csv",index_col=0)

In [69]:
web_scrapying_df_location=web_scrapying_df[['lat','lng']]

In [64]:
matrix_list=[]
for x in itertools.product(banch_mark_df.drop_duplicates().reset_index(drop=True).iterrows(),Gmap_df_location.reset_index(drop=True).iterrows()):
    #print(x)
    pair={
        'correct_id':x[0][0],
        'Gmap_id':x[1][0],
        "correct_geo":x[0][1][['lat','lon']].to_list(),
        "Gmap_geo":x[1][1][['place_lat','place_long']].to_list()
    }
    #print(pair)
    pair['distance']=calculate_distance(pair['correct_geo'],pair['Gmap_geo'])
    #print(pair)
    matrix_list.append(pair)

In [65]:
GMap_table=pd.DataFrame(matrix_list)

In [66]:
for threshold in np.arange(0.0,0.5,0.05):
    print(str(sum(GMap_table['distance']<=threshold))+" of pairs with distance within "+str(threshold)+' KM')

49 of pairs with distance within 0.0 KM
195 of pairs with distance within 0.05 KM
210 of pairs with distance within 0.1 KM
211 of pairs with distance within 0.15000000000000002 KM
214 of pairs with distance within 0.2 KM
214 of pairs with distance within 0.25 KM
214 of pairs with distance within 0.30000000000000004 KM
220 of pairs with distance within 0.35000000000000003 KM
225 of pairs with distance within 0.4 KM
229 of pairs with distance within 0.45 KM


In [67]:
GMap_pivot=GMap_table.pivot(index="correct_id",columns='Gmap_id',values="distance")

In [70]:
matrix_list=[]
for x in itertools.product(banch_mark_df.drop_duplicates().reset_index(drop=True).iterrows(),web_scrapying_df_location.drop_duplicates().reset_index(drop=True).iterrows()):
    #print(x)
    pair={
        'correct_id':x[0][0],
        'Web_id':x[1][0],
        "correct_geo":x[0][1][['lat','lon']].to_list(),
        "Web_geo":x[1][1].to_list()
    }
    pair['distance']=calculate_distance(pair['correct_geo'],pair['Web_geo'])
    #print(pair)
    matrix_list.append(pair)

In [71]:
web_table=pd.DataFrame(matrix_list)

In [72]:
for threshold in np.arange(0.0,0.5,0.05):
    print(str(sum(web_table['distance']<=threshold))+" of pairs with distance within "+str(threshold)+' KM')

118 of pairs with distance within 0.0 KM
253 of pairs with distance within 0.05 KM
259 of pairs with distance within 0.1 KM
261 of pairs with distance within 0.15000000000000002 KM
262 of pairs with distance within 0.2 KM
263 of pairs with distance within 0.25 KM
264 of pairs with distance within 0.30000000000000004 KM
264 of pairs with distance within 0.35000000000000003 KM
264 of pairs with distance within 0.4 KM
264 of pairs with distance within 0.45 KM


In [73]:
Web_pivot=web_table.pivot(index="correct_id",columns='Web_id',values="distance")

GMap performance

True => Found 1 to 1

In [74]:
sum_of_121=((GMap_pivot.T<0.1).sum(axis=0)==1).sum()
total=(GMap_pivot.T<0.1).sum(axis=0).count()
print("There are "+str(sum_of_121)+ " out of "+str(total)+ " with probation "+ str(100*sum_of_121/total)+"%")

There are 177 out of 283 with probation 62.54416961130742%


Found => True 1 to 1

In [75]:
sum_of_121=((GMap_pivot<0.1).sum(axis=0)==1).sum()
total=(GMap_pivot<0.1).sum(axis=0).count()
print("There are "+str(sum_of_121)+ " out of "+str(total)+ " with probation "+ str(100*sum_of_121/total)+"%")

There are 100 out of 289 with probation 34.602076124567475%


Web scraping performance

True => Found 1 to 1

In [76]:
sum_of_121=((Web_pivot.T<0.1).sum(axis=0)==1).sum()
total=(Web_pivot.T<0.1).sum(axis=0).count()
print("There are "+str(sum_of_121)+ " out of "+str(total)+ " with probation "+ str(100*sum_of_121/total)+"%")

There are 259 out of 283 with probation 91.51943462897526%


Found => True 1 to 1

In [77]:
sum_of_121=((Web_pivot<0.1).sum(axis=0)==1).sum()
total=(Web_pivot<0.1).sum(axis=0).count()
print("There are "+str(sum_of_121)+ " out of "+str(total)+ " with probation "+ str(100*sum_of_121/total)+"%")

There are 131 out of 195 with probation 67.17948717948718%


Evaluate after merging

GMap performance

In [85]:
matrix_list=[]
for x in itertools.product(merged_true.drop_duplicates().reset_index(drop=True).iterrows(),Gmap_df_location.reset_index(drop=True).iterrows()):
    #print(x)
    pair={
        'correct_id':x[0][0],
        'Gmap_id':x[1][0],
        "correct_geo":x[0][1][['lat','lon']].to_list(),
        "Gmap_geo":x[1][1][['place_lat','place_long']].to_list()
    }
    pair['distance']=calculate_distance(pair['correct_geo'],pair['Gmap_geo'])
    #print(pair)
    matrix_list.append(pair)

In [86]:
GMap_table=pd.DataFrame(matrix_list)
GMap_pivot=GMap_table.pivot(index="correct_id",columns='Gmap_id',values="distance")

True => Found 1 to 1

In [87]:
sum_of_121=((GMap_pivot.T<0.1).sum(axis=0)==1).sum()
total=(GMap_pivot.T<0.1).sum(axis=0).count()
print("There are "+str(sum_of_121)+ " out of "+str(total)+ " with probation "+ str(100*sum_of_121/total)+"%")

There are 129 out of 210 with probation 61.42857142857143%


Found => True 1 to 1

In [88]:
sum_of_121=((GMap_pivot<0.1).sum(axis=0)==1).sum()
total=(GMap_pivot<0.1).sum(axis=0).count()
print("There are "+str(sum_of_121)+ " out of "+str(total)+ " with probation "+ str(100*sum_of_121/total)+"%")

There are 150 out of 289 with probation 51.90311418685121%


GMap performance merged GMap dataset

In [89]:
matrix_list=[]
for x in itertools.product(merged_true.drop_duplicates().reset_index(drop=True).iterrows(),merged_Gmap.reset_index(drop=True).iterrows()):
    #print(x)
    pair={
        'correct_id':x[0][0],
        'Gmap_id':x[1][0],
        "correct_geo":x[0][1][['lat','lon']].to_list(),
        "Gmap_geo":x[1][1][['place_lat','place_long']].to_list()
    }
    pair['distance']=calculate_distance(pair['correct_geo'],pair['Gmap_geo'])
    #print(pair)
    matrix_list.append(pair)

In [90]:
GMap_merged_table=pd.DataFrame(matrix_list)
GMap_merged_pivot=GMap_merged_table.pivot(index="correct_id",columns='Gmap_id',values="distance")

True => Found 1 to 1

In [91]:
sum_of_121=((GMap_merged_pivot.T<0.1).sum(axis=0)==1).sum()
total=(GMap_merged_pivot.T<0.1).sum(axis=0).count()
print("There are "+str(sum_of_121)+ " out of "+str(total)+ " with probation "+ str(100*sum_of_121/total)+"%")

There are 139 out of 210 with probation 66.19047619047619%


Found => True 1 to 1

In [92]:
sum_of_121=((GMap_merged_pivot<0.1).sum(axis=0)==1).sum()
total=(GMap_merged_pivot<0.1).sum(axis=0).count()
print("There are "+str(sum_of_121)+ " out of "+str(total)+ " with probation "+ str(100*sum_of_121/total)+"%")

There are 139 out of 273 with probation 50.91575091575091%


Web scraping performance

In [93]:
matrix_list=[]
for x in itertools.product(merged_true.drop_duplicates().reset_index(drop=True).iterrows(),web_scrapying_df_location.drop_duplicates().reset_index(drop=True).iterrows()):
    #print(x)
    pair={
        'correct_id':x[0][0],
        'Web_id':x[1][0],
        "correct_geo":x[0][1][['lat','lon']].to_list(),
        "Web_geo":x[1][1].to_list()
    }
    pair['distance']=calculate_distance(pair['correct_geo'],pair['Web_geo'])
    #print(pair)
    matrix_list.append(pair)

In [94]:
web_table=pd.DataFrame(matrix_list)

In [95]:
Web_pivot=web_table.pivot(index="correct_id",columns='Web_id',values="distance")

True => Found 1 to 1

In [96]:
sum_of_121=((Web_pivot.T<0.1).sum(axis=0)==1).sum()
total=(Web_pivot.T<0.1).sum(axis=0).count()
print("There are "+str(sum_of_121)+ " out of "+str(total)+ " with probation "+ str(100*sum_of_121/total)+"%")

There are 191 out of 210 with probation 90.95238095238095%


Found => True

In [97]:
sum_of_121=((Web_pivot<0.1).sum(axis=0)==1).sum()
total=(Web_pivot<0.1).sum(axis=0).count()
print("There are "+str(sum_of_121)+ " out of "+str(total)+ " with probation "+ str(100*sum_of_121/total)+"%")

There are 191 out of 195 with probation 97.94871794871794%


This proves that the web scraping approach is more accurate